<a href="https://colab.research.google.com/github/kunai-3txk/Compe_tonyobyo/blob/main/AMP_fit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [1]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 11.7 MB/s eta 0:00:00


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_profiling
import os
import xgboost as xgb
import seaborn as sns; sns.set()

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import mean_squared_error


import lightgbm as lgb #LightGBM
from lightgbm import LGBMRegressor

from catboost import CatBoostRegressor,Pool

# Configuration

In [18]:
class config:
    METRIC = 'RMSE' #RMSE or SMAPE
    RANDOM_STATE=100  
    FRAC = 0.2
    N_FOLD = 5
    
class paths:
    # kaggle環境ならTrue
    if 'KAGGLE_URL_BASE' in set(os.environ.keys()):
        common_path = "/kaggle/input/amp-parkinsons-disease-progression-prediction"
    
    # colaboratory環境ならTrue
    if 'COLAB_GPU' in set(os.environ.keys()):
        common_path = "/content/drive/MyDrive/AMP"
    
    SAMPLE_SUBMISSION = common_path + "/example_test_files/sample_submission.csv"
    SUPPLEMENTAL_CLINICAL_DATA = common_path + "/supplemental_clinical_data.csv"
    TRAIN_CLINICAL_DATA = common_path + "/train_clinical_data.csv"
    TRAIN_PEPTIDES = common_path + "/train_peptides.csv"
    TRAIN_PROTEINS = common_path + "/train_proteins.csv"
    TEST_CLINICAL_DATA = common_path + "/example_test_files/test.csv"
    TEST_PEPTIDES = common_path + "/example_test_files/test_peptides.csv"
    TEST_PROTEINS = common_path + "/example_test_files/test_proteins.csv"
    #train output
    TRAIN = common_path + "/train_all.csv"
    TEST = common_path + "/test_all.csv"
    TE_UPDRS_1 = common_path + "/train_te_updrs_1.csv"
    TE_UPDRS_2 = common_path + "/train_te_updrs_2.csv"
    TE_UPDRS_3 = common_path + "/train_te_updrs_3.csv"
    TE_UPDRS_4 = common_path + "/train_te_updrs_4.csv"

# Function Definition

## display_feature_importance

In [4]:
def display_feature_importance(feature_list,model):

    # 特徴量重要度を保管する dataframe を用意
    feature_importance = pd.DataFrame()
    feature_importance['feature'] = feature_list
    feature_importance['importance'] = model.feature_importances_

    print('****************************') 
    display(feature_importance.groupby("feature")["importance"].mean().sort_values(ascending=False)[:20])

    # order = list(feature_importance.groupby("feature")["importance"].mean().sort_values(ascending=False).index)[:30]
    # # 可視化
    # plt.figure(figsize=(10, 10))
    # sns.barplot(x="importance",y="feature",data=feature_importance,order=order)
    # plt.title('importance')
    # plt.tight_layout()
    # plt.show()

## score_cal

In [5]:
def score_cal(y_true, y_pred,metric = config.METRIC):
  if metric == 'SMAPE':
    smap = np.zeros(len(y_true))
    num = np.abs(y_true - y_pred)
    dem = ((np.abs(y_true) + np.abs(y_pred)) / 2)
    pos_ind = (y_true!=0)|(y_pred!=0)
    smap[pos_ind] = num[pos_ind] / dem[pos_ind]
    ret = 100 * np.mean(smap)
  if metric == 'RMSE':
    ret = np.sqrt(mean_squared_error(y_true,y_pred))

  return ret

## fit_catboost_fold1

In [6]:
def fit_catboost_fold1(X,y):

  X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=config.FRAC, random_state=config.RANDOM_STATE,shuffle=True)

  train_pool = Pool(X_train, y_train)
  model = CatBoostRegressor(random_seed=config.RANDOM_STATE,logging_level='Silent',eval_metric = config.METRIC)
  model.fit(train_pool,plot=False, verbose=False)  

  test_score = score_cal(y_val, model.predict(X_val))
  print('****************************')
  print(f"Test data average score : {test_score:.4f}"," seed ", config.RANDOM_STATE)

  return model

# Load Data

## train

In [7]:
df_train = pd.read_csv(paths.TRAIN)
print('train shape:',df_train.shape)

te_updrs_1 = pd.read_csv(paths.TE_UPDRS_1)
te_updrs_2 = pd.read_csv(paths.TE_UPDRS_2)
te_updrs_3 = pd.read_csv(paths.TE_UPDRS_3)
te_updrs_4 = pd.read_csv(paths.TE_UPDRS_4)


train shape: (2588, 104)


## test

In [39]:
df_test = pd.read_csv(paths.TEST)

test = pd.read_csv(paths.TEST_CLINICAL_DATA)
test_peptides = pd.read_csv(paths.TEST_PEPTIDES)
test_proteins = pd.read_csv(paths.TEST_PROTEINS)
sample_submission = pd.read_csv(paths.SAMPLE_SUBMISSION)
print('Proteins shape:',test_proteins.shape)
print('Peptides shape:',test_peptides.shape)
print('Clinical shape:',test.shape)

,visit_id,visit_month,patient_id,updrs_1_mean_by_month,updrs_2_mean_by_month,updrs_3_mean_by_month,updrs_4_mean_by_month,updrs_1_std_by_month,updrs_2_std_by_month,updrs_3_std_by_month,...,updrs_1_median_by_Pep_std_by_id,updrs_2_median_by_Pep_std_by_id,updrs_3_median_by_Pep_std_by_id,updrs_4_median_by_Pep_std_by_id,upd23b_clinical_state_on_medication_median_by_Pep_std_by_id,updrs_1_max_by_Pep_std_by_id,updrs_2_max_by_Pep_std_by_id,updrs_3_max_by_Pep_std_by_id,updrs_4_max_by_Pep_std_by_id,upd23b_clinical_state_on_medication_max_by_Pep_std_by_id
0,3342_0,0,3342,5.725108,5.734848,18.312684,1.186517,4.452248,5.161743,12.375385,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,50423_0,0,50423,5.725108,5.734848,18.312684,1.186517,4.452248,5.161743,12.375385,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3342_6,6,3342,5.938636,6.670455,22.143836,1.427419,4.956150,5.233765,11.905418,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,50423_6,6,50423,5.938636,6.670455,22.143836,1.427419,4.956150,5.233765,11.905418,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Make Test_Val Data

In [9]:
#pivot
df_train_pv=df_train[['visit_id','patient_id','visit_month','updrs_1','updrs_2','updrs_3','updrs_4']].dropna(axis=0).pivot(index="patient_id",columns='visit_month',values='visit_id').reset_index()
df_train_pv.head()

#testデータ用の候補探し
test_patient_candidate_list = list(df_train_pv[
    ~df_train_pv[0].isna() & ~df_train_pv[6].isna() & ~df_train_pv[12].isna() & ~df_train_pv[24].isna()
]['patient_id'].values)
len(test_patient_candidate_list)


56

In [24]:
#test_patient_list = [1517,3863] #決め打ち　Top1,2
test_patient_list = [4923,7265] #決め打ち  Top3,4

test_month_list = [0,6,12,24]   #TESTと同一
#test_month_list = [0,3,6,9,12,18,24]
df_test_val = df_train[df_train['patient_id'].isin(test_patient_list) & df_train['visit_month'].isin(test_month_list)].reset_index(drop=True)

df_test_val_true = pd.DataFrame()

for i in range(0,3):
  df_test_val_true_ = pd.DataFrame()
  df_test_val_true_['id']=df_test_val[['visit_id']] + '_updrs_{0}'.format(i+1)
  df_test_val_true_['updrs']=df_test_val[['updrs_{0}'.format(i+1)]]
  df_test_val_true = pd.concat([df_test_val_true,df_test_val_true_],axis=0)

,id,updrs
0,4923_0_updrs_1,2.0
1,4923_6_updrs_1,6.0
2,4923_12_updrs_1,6.0
3,4923_24_updrs_1,4.0
4,7265_0_updrs_1,10.0
5,7265_6_updrs_1,12.0
6,7265_12_updrs_1,12.0
7,7265_24_updrs_1,15.0
0,4923_0_updrs_2,3.0
1,4923_6_updrs_2,3.0


# Fit

## fit main

In [11]:

#predict_order_list = [2,1,3]
predict_order_list = [1,2,3]
#predict_order_list = [3,1,2]
#predict_order_list = [1]

df_train_ = df_train.drop(['visit_id','patient_id','updrs_1','updrs_2','updrs_3','updrs_4','upd23b_clinical_state_on_medication'],axis=1)

for order in predict_order_list:
  
  train_X = df_train_.values
  train_y = df_train['updrs_{0}'.format(order)].values
  model = fit_catboost_fold1(train_X,train_y)

  if order == 1:
        model_1 = model
        df_te = te_updrs_1
  if order == 2:
        model_2 = model
        df_te = te_updrs_2
  if order == 3:
        model_3 = model
        df_te = te_updrs_3

  #display importance
  display_feature_importance(df_train_.columns,model)

  #予測値をtrainにセット
  df_train_['updrs_{0}'.format(order)] = model.predict(train_X)
  df_train_['updrs_{0}_round'.format(order)] = df_train_['updrs_{0}'.format(order)].round(0)
  #予測値のtarget-encodingテーブルをjoin
  df_te.rename(columns={'updrs_{0}'.format(order):'updrs_{0}_te'.format(order)},inplace=True)
  df_train_ = df_train_.merge(df_te,left_on='updrs_{0}_round'.format(order),right_on='updrs_{0}_te'.format(order),how='left')
  #不要列の削除
  df_train_.drop(['updrs_{0}_round'.format(order),'updrs_{0}_te'.format(order)],axis=1,inplace=True)
  print('train shape:',df_train_.shape)


****************************
Test data average score : 3.7156  seed  100
****************************


feature
updrs_1_std_by_Pep_mean_by_id                                  5.950356
updrs_2_std_by_Pep_mean_by_id                                  4.069412
updrs_1_median_by_Pep_std_by_id                                3.479150
updrs_1_median_by_Pep_mean_by_id                               2.628452
updrs_2_max_by_Pep_mean_by_id                                  2.421334
updrs_2_median_by_Pep_std_by_id                                2.409764
Abu_median_by_id                                               2.026531
updrs_1_max_by_Pep_std_by_id                                   2.003564
updrs_4_max_by_Pep_std_by_id                                   1.994635
updrs_1_max_by_Uni_std_by_id                                   1.960829
updrs_4_max_by_Uni_std_by_id                                   1.925439
updrs_1_max_by_Uni_mean_by_id                                  1.873434
updrs_1_mean_by_Pep_mean_by_id                                 1.616220
updrs_2_max_by_Uni_mean_by_id                           

train shape: (2588, 107)
****************************
Test data average score : 3.7886  seed  100
****************************


feature
updrs_1                                                       10.010614
updrs_3_mean_by_updrs_1                                        4.163433
updrs_2_std_by_Pep_mean_by_id                                  3.378742
updrs_2_mean_by_updrs_1                                        2.944793
updrs_3_std_by_Pep_mean_by_id                                  2.895400
updrs_2_median_by_updrs_1                                      2.868160
updrs_3_median_by_updrs_1                                      2.478600
NPX_median_by_id                                               2.420435
updrs_2_std_by_Uni_mean_by_id                                  2.115327
updrs_4_mean_by_updrs_1                                        1.982686
updrs_2_median_by_Pep_std_by_id                                1.914727
upd23b_clinical_state_on_medication_mean_by_Uni_mean_by_id     1.888742
updrs_3_std_by_Uni_mean_by_id                                  1.575750
updrs_2_max_by_Uni_mean_by_id                           

train shape: (2588, 117)
****************************
Test data average score : 9.5150  seed  100
****************************


feature
updrs_2                                                        13.965420
updrs_1_mean_by_updrs_2                                         5.395815
updrs_3_mean_by_updrs_2                                         4.522386
updrs_1_median_by_updrs_2                                       3.810024
updrs_3_median_by_updrs_2                                       3.802154
updrs_3_std_by_Pep_mean_by_id                                   2.722465
updrs_4_mean_by_updrs_2                                         2.495148
updrs_3_std_by_Uni_mean_by_id                                   1.995329
updrs_1                                                         1.813549
NPX_median_by_id                                                1.716904
updrs_2_median_by_Pep_std_by_id                                 1.634839
Abu_median_by_id                                                1.629126
updrs_4_max_by_Pep_std_by_id                                    1.561189
upd23b_clinical_state_on_medication_mean_by

train shape: (2588, 127)


## predict Test_Val

In [27]:
df_test_val_ = df_test_val.drop(['visit_id','patient_id','updrs_1','updrs_2','updrs_3','updrs_4','upd23b_clinical_state_on_medication'],axis=1)

for order in predict_order_list:
  test_X = df_test_val_.values
  test_y = df_test_val['updrs_{0}'.format(order)].values
  if order == 1:
        model_ = model_1
        df_te_val = te_updrs_1
  if order == 2:
        model_ = model_2
        df_te_val = te_updrs_2
  if order == 3:
        model_ = model_3
        df_te_val = te_updrs_3

  df_test_val_['updrs_{0}'.format(order)] = model_.predict(test_X)
  df_test_val_['updrs_{0}_round'.format(order)] = df_test_val_['updrs_{0}'.format(order)].round(0)
  #予測値のtarget-encodingテーブルをjoin
  df_te_val.rename(columns={'updrs_{0}'.format(order):'updrs_{0}_te'.format(order)},inplace=True)
  df_test_val_ = df_test_val_.merge(df_te_val,left_on='updrs_{0}_round'.format(order),right_on='updrs_{0}_te'.format(order),how='left')
  #不要列の削除
  df_test_val_.drop(['updrs_{0}_round'.format(order),'updrs_{0}_te'.format(order)],axis=1,inplace=True)

df_test_val_pred_ = df_test_val[['visit_id']].join(df_test_val_[['updrs_1','updrs_2','updrs_3']])

df_test_val_pred = pd.DataFrame()
for i in range(0,3):
  df_test_val_pred_vs = pd.DataFrame()
  df_test_val_pred_vs['id'] = df_test_val_pred_[['visit_id']] + '_updrs_{0}'.format(i+1)
  df_test_val_pred_vs['updrs']=df_test_val_pred_[['updrs_{0}'.format(i+1)]]
  df_test_val_pred = pd.concat([df_test_val_pred,df_test_val_pred_vs],axis=0)

print('****************************')
print('Score Predict : ',score_cal(df_test_val_true['updrs'].values,df_test_val_pred['updrs'].values))
print('Score Predict SMAPE : ',score_cal(df_test_val_true['updrs'].values,df_test_val_pred['updrs'].values,'SMAPE'))

****************************
Score Predict :  9.615585009145715
Score Predict SMAPE :  38.80301128107919


## display Test_Val True vs Pred

In [26]:
#どれぐらい差があるか調べる
df_test_val_truepred = df_test_val_true.merge(df_test_val_pred,on='id',how='left').rename(columns={'updrs_x':'true','updrs_y':'pred'}).round(1)
df_test_val_truepred['diff'] = abs(df_test_val_truepred['true']-df_test_val_truepred['pred'])
df_test_val_truepred['diff_ratio'] = round((df_test_val_truepred['diff'] / df_test_val_truepred['true']) * 100,0)
display(df_test_val_truepred)

,id,true,pred,diff,diff_ratio
0,4923_0_updrs_1,2.0,6.8,4.8,240.0
1,4923_6_updrs_1,6.0,10.1,4.1,68.0
2,4923_12_updrs_1,6.0,5.9,0.1,2.0
3,4923_24_updrs_1,4.0,5.5,1.5,38.0
4,7265_0_updrs_1,10.0,6.8,3.2,32.0
5,7265_6_updrs_1,12.0,10.1,1.9,16.0
6,7265_12_updrs_1,12.0,13.6,1.6,13.0
7,7265_24_updrs_1,15.0,14.6,0.4,3.0
8,4923_0_updrs_2,3.0,6.4,3.4,113.0
9,4923_6_updrs_2,3.0,7.4,4.4,147.0


## predict Test for colab

In [40]:
df_test_ = df_test.drop(['visit_id'],axis=1)

for order in predict_order_list:
  test_X = df_test_.values
  #test_y = df_test['updrs_{0}'.format(order)].values
  if order == 1:
        model_ = model_1
        df_te = te_updrs_1
  if order == 2:
        model_ = model_2
        df_te = te_updrs_2
  if order == 3:
        model_ = model_3
        df_te = te_updrs_3

  df_test_['updrs_{0}'.format(order)] = model_.predict(test_X)
  df_test_['updrs_{0}_round'.format(order)] = df_test_['updrs_{0}'.format(order)].round(0)
  #予測値のtarget-encodingテーブルをjoin
  df_te.rename(columns={'updrs_{0}'.format(order):'updrs_{0}_te'.format(order)},inplace=True)
  df_test_ = df_test_.merge(df_te,left_on='updrs_{0}_round'.format(order),right_on='updrs_{0}_te'.format(order),how='left')
  #不要列の削除
  df_test_.drop(['updrs_{0}_round'.format(order),'updrs_{0}_te'.format(order)],axis=1,inplace=True)

df_test_pred_ = df_test[['visit_id']].join(df_test_[['updrs_1','updrs_2','updrs_3']])

df_test_pred = pd.DataFrame()
for i in range(0,3):
  df_test_pred_vs = pd.DataFrame()
  df_test_pred_vs['id'] = df_test_pred_[['visit_id']] + '_updrs_{0}'.format(i+1)
  df_test_pred_vs['updrs']=df_test_pred_[['updrs_{0}'.format(i+1)]]
  df_test_pred = pd.concat([df_test_pred,df_test_pred_vs],axis=0)

display(df_test_pred)

,id,updrs
0,3342_0_updrs_1,8.336880
1,50423_0_updrs_1,6.489496
2,3342_6_updrs_1,6.475163
3,50423_6_updrs_1,6.475163
0,3342_0_updrs_2,7.003556
1,50423_0_updrs_2,5.935182
2,3342_6_updrs_2,5.935182
3,50423_6_updrs_2,5.935182
0,3342_0_updrs_3,10.396216
1,50423_0_updrs_3,9.025155


# Inference for kaggle notebook

## import Library

In [14]:
if 'KAGGLE_URL_BASE' in set(os.environ.keys()):
  import amp_pd_peptide
  env = amp_pd_peptide.make_env()
  iter_test = env.iter_test()

In [15]:
# def map_test(x):
#     updrs = x.split('_')[2] + '_' + x.split('_')[3]
#     month = int(x.split('_plus_')[1].split('_')[0])
#     visit_id = x.split('_')[0] + '_' + x.split('_')[1]
#     if updrs=='updrs_4':
#         rating = 0
#     elif updrs =='updrs_1':
#         rating = df[df.visit_id == visit_id]['pred1'].values[0]
#     elif updrs =='updrs_2':
#         rating = df[df.visit_id == visit_id]['pred2'].values[0]
#     else: #updrs =='updrs_3'
#         rating = df[df.visit_id == visit_id]['pred3'].values[0]
#     return rating

if 'KAGGLE_URL_BASE' in set(os.environ.keys()):
  counter = 0

  for (test, test_peptides, test_proteins, sample_submission) in iter_test:

    #1.common 特徴量の作成

    #2-1.predict to updrs_1

    #2-2.updrs_1を使った追加の特徴量作成

    #3-1.predict to updrs_2

    #3-2.updrs_1を使った追加の特徴量作成

    #4.predict to updrs_3
    
    #5.submissionの作成

    sample_submission['rating'] = sample_submission['prediction_id'].apply(map_test)
    env.predict(sample_submission)
    
    if counter == 0:
        display(test)
        display(sample_submission)
        
    counter += 1

In [16]:

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=config.FRAC, random_state=config.RANDOM_STATE,shuffle=True)

# df_log = pd.DataFrame() #Log用DFの定義
# models = []                          # 各 fold のモデル
# scores = 0.0   
# best_score = np.inf
# preds = np.zeros(len(X_test))

# gf = GroupShuffleSplit(n_splits=config.N_FOLD,test_size=0.2,random_state=config.RANDOM_STATE)
# groups = X_train[:,0]

# for fold, (trn_idx, val_idx) in enumerate(gf.split(X_train,y_train,groups)):

#     X_trn, y_trn = X_train[trn_idx,:], y_train[trn_idx]
#     X_val, y_val = X_train[val_idx,:], y_train[val_idx]

#     train_pool = Pool(X_trn, y_trn)
#     validate_pool = Pool(X_val, y_val)

#     model = CatBoostRegressor(random_seed=config.RANDOM_STATE,logging_level='Silent',eval_metric = config.METRIC)
#     model.fit(train_pool, eval_set = validate_pool, early_stopping_rounds = 10, use_best_model=True, plot=False, verbose=False)  
    
#     score = score_cal(y_val,model.predict(X_val))
#     scores += score / config.N_FOLD
#     models.append(model)

#     if score < best_score:
#         best_score = score
#         best_model = model
#         best_fold = fold+1
#     print("Fold :" , fold+1)

# print(f"fold average score : {scores:.4f}")
# print(f"fold best score : {best_score:.4f}"," FOLD ",best_fold)
# #****************************
# # test データ

# # 各モデルで推論
# for model in models:
#     preds += model.predict(X_test) / len(models) 
# test_score = score_cal(y_test, preds)
# pred_best = best_model.predict(X_test)

# print(f"Test data average score : {test_score:.4f}"," seed ", config.RANDOM_STATE)
# print(f"Test data best socre : {score_cal(y_test, pred_best):.4f}"," seed ", config.RANDOM_STATE)
# print("***********************")
# #display_feature_importance(train_column_list,best_model)
# feature_importance = pd.DataFrame()
# feature_importance['feature'] = train_column_list
# feature_importance['importance'] = best_model.feature_importances_

# feature_importance.groupby("feature")["importance"].mean().sort_values(ascending=False)[:30]